In [1]:
#Install Dependencies and SkTime
!pip install tensorflow
!pip install torch
!pip install sktime\[dl\]
!pip install sktime\[all_extras\]

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sktime
import torch
from sktime.datasets import load_basic_motions
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import OneHotEncoder
from sktime.classification.deep_learning import MVTSTransformerClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import LeaveOneGroupOut

In [81]:
import torch
print(torch.__version__)

2.5.1


In [82]:
import sktime
print(sktime.__version__)

0.34.1


In [3]:
from sktime.classifiers.deep_learning import MVTSTransformerClassifier

ModuleNotFoundError: No module named 'sktime.classifiers'

data preparation
Shape: (num_samples, timesteps, num_features)

In [2]:
data_path = "../data"

In [4]:
imu_df=pd.read_csv(f'{data_path}/imu.csv')

In [5]:
imu_df.head()

,participant,body_part,condition,task,file_path,PacketCounter,SampleTimeFine,Quat_W,Quat_X,Quat_Y,...,Acc_X,Acc_Y,Acc_Z,Gyr_X,Gyr_Y,Gyr_Z,Mag_X,Mag_Y,Mag_Z,Unnamed: 15
0,P_02,arm_l,natural,Cupplacing,/Users/daikexin/pythonProject/CompensatoryMove...,0,1970-01-01 21:07:25,-0.736210,0.183717,0.638716,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.262207,0.231201,-0.031738,
1,P_02,arm_l,natural,Cupplacing,/Users/daikexin/pythonProject/CompensatoryMove...,1,1970-01-02 01:45:12,-0.736222,0.183720,0.638712,...,9.708877,-1.067807,1.129502,-1.429176,0.690589,0.529434,-0.262695,0.232666,-0.031250,
2,P_02,arm_l,natural,Cupplacing,/Users/daikexin/pythonProject/CompensatoryMove...,2,1970-01-02 06:22:59,-0.736170,0.183847,0.638716,...,9.700046,-1.093796,1.111337,-1.696598,0.339913,0.393840,-0.263672,0.232178,-0.031250,
3,P_02,arm_l,natural,Cupplacing,/Users/daikexin/pythonProject/CompensatoryMove...,3,1970-01-02 11:00:46,-0.736107,0.183970,0.638727,...,9.719351,-1.097965,1.145534,-1.721801,0.291210,0.197749,-0.262207,0.233643,-0.032959,
4,P_02,arm_l,natural,Cupplacing,/Users/daikexin/pythonProject/CompensatoryMove...,4,1970-01-02 15:38:33,-0.736076,0.183974,0.638764,...,9.745931,-1.099311,1.121240,-1.277600,0.308383,0.286817,-0.260254,0.232666,-0.030273,


Leave-One-Participant-Out Cross-Validation

In [60]:
imu=imu_df.drop(columns=['file_path', 'SampleTimeFine'])

In [61]:
participants = imu['participant']
labels = imu['condition']  
features = imu.drop(columns=['participant', 'condition', 'task'])  # Drop non-feature columns

In [62]:
# Convert features and labels into NumPy arrays
X = features.to_numpy()
y = labels.to_numpy()

In [63]:
# One-Hot Encode 
encoder = OneHotEncoder(sparse_output=False)
metadata = imu[['body_part', 'task']].to_numpy()
metadata_encoded = encoder.fit_transform(metadata)
X_with_metadata = np.hstack((X, metadata_encoded))

In [64]:
X_with_metadata

array([['arm_l', 0, -0.7362104058265686, ..., 0.0, 0.0, 0.0],
       ['arm_l', 1, -0.7362218499183655, ..., 0.0, 0.0, 0.0],
       ['arm_l', 2, -0.7361699938774109, ..., 0.0, 0.0, 0.0],
       ...,
       ['wrist_r', 1242, 0.9784764051437378, ..., 0.0, 0.0, 1.0],
       ['wrist_r', 1243, 0.9785610437393188, ..., 0.0, 0.0, 1.0],
       ['wrist_r', 1244, 0.9786368012428284, ..., 0.0, 0.0, 1.0]],
      dtype=object)

In [65]:
# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(imu['condition'])

In [66]:
# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

In [80]:
# Initialize transformer classifier
mvts = MVTSTransformerClassifier(num_epochs=20, batch_size=8)

ModuleNotFoundError: MVTSTransformerClassifier requires package 'torch' to be present in the python environment, but 'torch' was not found. 'torch' is a soft dependency and not included in the base sktime installation. Please run: `pip install torch` to install the torch package. To install all soft dependencies, run: `pip install sktime[all_extras]`

In [ ]:
# Perform LOPO-CV
accuracies = []
for train_idx, test_idx in logo.split(X_with_metadata, y_encoded, groups=participants):
    # Split data
    X_train, X_test = X_with_metadata[train_idx], X_with_metadata[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]
    
    # Fit model
    mvts.fit(X_train, y_train)
    
    # Evaluate on test set
    accuracy = mvts.score(X_test, y_test)
    accuracies.append(accuracy)
    print(f"Participant Left Out: {participants[test_idx][0]}, Test Accuracy: {accuracy:.2f}")

# Average accuracy
print(f"Mean LOPO-CV Accuracy: {np.mean(accuracies):.2f}")

In [ ]:
# LOPO-CV
logo = LeaveOneGroupOut()
cv_results = []

for train_idx, test_idx in logo.split(X_with_metadata, y, groups=participants):
    X_train, X_test = X_with_metadata[train_idx], X_with_metadata[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Build and Train the Model
    model = build_rnn_model()  # Replace with your model-building function
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Evaluate the Model
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    cv_results.append(test_acc)

# Compute Average Accuracy
average_accuracy = np.mean(cv_results)
print(f"LOPO-CV Average Accuracy: {average_accuracy * 100:.2f}%")